In [8]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

In [9]:
#1 Wide Receiver
df = pd.read_csv("../WR/wrstats3.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,drafted,Crec,Cyds,CTD,hasbling,allamerican,hasawards
0,2017,jalen robinette,Air Force,WR,74.75,220,4.62,13.0,31.5,120.0,4.46,6.77,N,120.0,2697.0,18.0,0,0,0
1,2017,jerome lane,Akron,WR,74.13,226,4.60,16.0,35.0,125.0,4.12,6.87,N,101.0,1800.0,14.0,0,0,0
2,2002,jake schifino,Akron,WR,72.60,200,4.47,NaN,NaN,NaN,NaN,NaN,Y,131.0,1986.0,10.0,0,0,0
3,2018,robert foster,Alabama,WR,74.00,196,4.41,NaN,31.0,116.0,4.20,6.90,N,35.0,389.0,3.0,0,0,0
4,2002,jason mcaddley,Alabama,WR,73.50,203,4.50,NaN,36.5,120.0,4.09,6.83,Y,71.0,1020.0,8.0,0,0,0


In [10]:
Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

In [11]:
#1st iteration
X["benchpress"] = X["benchpress"].fillna(value=0)
X["vert_leap_in"] = X["vert_leap_in"].fillna(value=0)
X["broad_jump_in"] = X["broad_jump_in"].fillna(value=0)
X["Crec"] = X["Crec"].fillna(value=0)
X["Cyds"] = X["Cyds"].fillna(value=0)
X["CTD"] = X["CTD"].fillna(value=0)
X["shuttle"] = X["shuttle"].fillna(value=100)
X["3cone"] = X["3cone"].fillna(value=100)
X["40yards"] = X["40yards"].fillna(value=100)
X = X.drop([item for item in X.columns.tolist() if item not in ['40yards', 'height_in', 'benchpress', 'vert_leap_in', 'broad_jump_in', 'CTD',"hasawards"]], 1)
X.head()

,height_in,40yards,benchpress,vert_leap_in,broad_jump_in,CTD,hasawards
0,74.75,4.62,13.0,31.5,120.0,18.0,0
1,74.13,4.60,16.0,35.0,125.0,14.0,0
2,72.60,4.47,0.0,0.0,0.0,10.0,0
3,74.00,4.41,0.0,31.0,116.0,3.0,0
4,73.50,4.50,0.0,36.5,120.0,8.0,0


In [12]:
# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier(learning_rate=.1,n_estimators=400,max_depth=2,min_child_weight=1,gamma=.3,colsample_bytree=.7)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 75.62%


In [13]:
#Testing Data - Wide Receiver

In [27]:
#1 Wide Receiver
testdata = pd.read_csv("WR-Test.csv")
testdata

,name,college,position,height_in,40yards,benchpress,vert_leap_in,broad_jump_in,CTD,hasawards
0,"Boykin, Miles",Notre Dame,WR,76,4.42,12.0,43.5,140.0,11.0,0.0
1,"Brady, Tyre",Marshall,WR,75,NaN,12.0,32.0,117.0,18.0,0.0
2,"Brown, A.J.",Ole Miss,WR,72,4.49,19.0,36.5,120.0,NaN,0.0
3,"Butler, Emmanuel",Northern Arizona,WR,75,NaN,11.0,36.0,115.0,2.0,0.0
4,"Butler, Hakeem",Iowa St.,WR,77,4.48,18.0,36.0,128.0,18.0,0.0
5,"Campbell, Parris",Ohio St.,WR,72,4.31,11.0,40.0,135.0,15.0,0.0
6,"Custis, Jamal",Syracuse,WR,76,4.50,NaN,34.5,124.0,8.0,0.0
7,"Davis, Felton",Michigan St.,WR,75,NaN,21.0,NaN,NaN,14.0,0.0
8,"Davis, Ryan",Auburn,WR,70,NaN,12.0,35.0,124.0,7.0,0.0
9,"Dixon, Johnnie",Ohio St.,WR,70,4.41,16.0,37.5,120.0,16.0,0.0


In [28]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [29]:
for index, row in testdata.iterrows():
    if (testdata.loc[index,"hasawards"] != 1.0):
        if (testdata.loc[index,"hasawards"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                testdata.loc[index, "CTD"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rec_td']").text
                    
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    testdata.loc[index,"hasawards"] = 1
                else:
                    testdata.loc[index,"hasawards"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                testdata.loc[index,"hasawards"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            testdata.loc[index, "CTD"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rec_td']").text
                    
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                testdata.loc[index,"hasawards"] = 1
                            else:
                                testdata.loc[index,"hasawards"] = 0
                        else:
                            testdata.loc[index,"hasawards"] = 0
                            break
                    else:
                        testdata.loc[index,"hasawards"] = 0

In [30]:
testdata.to_csv("WR-Test.csv", index=False)

In [39]:
#Testing Data
testdata = pd.read_csv("WR-Test.csv")
Name = testdata["name"]
Position = testdata["position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["name","college","position"],1)
testdata.head()

,height_in,40yards,benchpress,vert_leap_in,broad_jump_in,CTD,hasawards
0,76,4.42,12.0,43.5,140.0,11.0,0.0
1,75,NaN,12.0,32.0,117.0,18.0,0.0
2,72,4.49,19.0,36.5,120.0,NaN,0.0
3,75,NaN,11.0,36.0,115.0,2.0,0.0
4,77,4.48,18.0,36.0,128.0,18.0,0.0


In [40]:
#testdata
testdata["benchpress"] = testdata["benchpress"].fillna(value=0)
testdata["vert_leap_in"] = testdata["vert_leap_in"].fillna(value=0)
testdata["broad_jump_in"] = testdata["broad_jump_in"].fillna(value=0)
testdata["CTD"] = testdata["CTD"].fillna(value=0)
testdata["40yards"] = testdata["40yards"].fillna(value=100)

In [41]:
testdata.dtypes

height_in          int64
40yards          float64
benchpress       float64
vert_leap_in     float64
broad_jump_in    float64
CTD              float64
hasawards        float64
dtype: object

In [42]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

In [61]:
Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")

In [62]:
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Miles Boykin,WR,1
1,Tyre Brady,WR,0
2,A.J. Brown,WR,1
3,Emmanuel Butler,WR,0
4,Hakeem Butler,WR,1
5,Parris Campbell,WR,1
6,Jamal Custis,WR,1
7,Felton Davis,WR,1
8,Ryan Davis,WR,0
9,Johnnie Dixon,WR,1


In [63]:
testingdata = pd.read_csv("../../all predictions draftsite.csv")
testingdata

,Name,Position,Predictions (first),Predictions #2,draftsite,draftwire,nflspinzone,model2,finalmodel,ACTUAL
0,A.J. Brown,WR,1,1,1.0,1.0,1,NaN,NaN,NaN
1,Albert Huggins,DL,1,1,NaN,0.0,0,NaN,NaN,NaN
2,Alec Ingold,FB,0,0,1.0,0.0,0,NaN,NaN,NaN
3,Alex Barnes,RB,1,1,1.0,1.0,0,NaN,NaN,NaN
4,Alex Wesley,WR,1,1,NaN,0.0,1,NaN,NaN,NaN
5,Alexander Mattison,RB,1,0,NaN,1.0,1,NaN,NaN,NaN
6,Alijah Holder,CB,1,1,0.0,0.0,0,NaN,NaN,NaN
7,Alize Mack,TE,1,1,1.0,0.0,0,NaN,NaN,NaN
8,Amani Hooker,S,1,1,1.0,1.0,1,NaN,NaN,NaN
9,Amani Oruwariye,CB,1,1,1.0,1.0,1,NaN,NaN,NaN


In [73]:
testingdata = testingdata.merge(predictors, on=["Name","Position"], how="left")

In [74]:
#2 Quarterbacks
df = pd.read_csv("../DL/DL-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

X = X.drop([item for item in X.columns.tolist() if item not in ['40yards', 'height_in', 'weight_lbs', 'broad_jump_in', 'shuttle',"allamerican"]], 1)

# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier(learning_rate=.1,n_estimators=100,max_depth=2,min_child_weight=3,gamma=0.1,colsample_bytree=.7)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 76.57%


In [82]:
testdata = pd.read_csv("DL-Test.csv")
testdata

,name,college,position,height_in,weight_lbs,40yards,broad_jump_in,shuttle,allamerican
0,"Allen, Josh",Kentucky,DL,77,262.0,4.63,112.0,4.36,1.0
1,"Banogu, Ben",TCU,DL,75,250.0,4.62,134.0,4.27,0.0
2,"Bosa, Nick",Ohio St.,DL,76,266.0,4.79,116.0,4.14,0.0
3,"Brailford, Jordan",Oklahoma St.,DL,75,252.0,4.65,126.0,NaN,0.0
4,"Burns, Brian",Florida St.,DL,77,249.0,4.53,129.0,NaN,0.0
5,"Carney, Malik",North Carolina,DL,74,251.0,4.73,113.0,NaN,0.0
6,"Crosby, Maxx",E. Michigan,DL,77,255.0,4.66,122.0,4.13,0.0
7,"Davis, Jamal",Akron,DL,75,243.0,4.60,122.0,4.44,0.0
8,"Ferrell, Clelin",Clemson,DL,76,264.0,NaN,NaN,4.40,1.0
9,"Granderson, Carl",Wyoming,DL,77,254.0,4.79,119.0,4.41,0.0


In [76]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [83]:
for index, row in testdata.iterrows():
    if (testdata.loc[index,"allamerican"] != 1.0):
        if (testdata.loc[index,"allamerican"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    testdata.loc[index,"allamerican"] = 1
                else:
                    testdata.loc[index,"allamerican"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                testdata.loc[index,"allamerican"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):

                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                testdata.loc[index,"allamerican"] = 1
                            else:
                                testdata.loc[index,"allamerican"] = 0
                            break
                        else:
                            testdata.loc[index,"allamerican"] = 0
                            break
                    else:
                        testdata.loc[index,"allamerican"] = 0


In [84]:
testdata.to_csv("DL-Test.csv", index=False)

In [85]:
testdata

,name,college,position,height_in,weight_lbs,40yards,broad_jump_in,shuttle,allamerican
0,"Allen, Josh",Kentucky,DL,77,262.0,4.63,112.0,4.36,1.0
1,"Banogu, Ben",TCU,DL,75,250.0,4.62,134.0,4.27,0.0
2,"Bosa, Nick",Ohio St.,DL,76,266.0,4.79,116.0,4.14,0.0
3,"Brailford, Jordan",Oklahoma St.,DL,75,252.0,4.65,126.0,NaN,0.0
4,"Burns, Brian",Florida St.,DL,77,249.0,4.53,129.0,NaN,0.0
5,"Carney, Malik",North Carolina,DL,74,251.0,4.73,113.0,NaN,0.0
6,"Crosby, Maxx",E. Michigan,DL,77,255.0,4.66,122.0,4.13,0.0
7,"Davis, Jamal",Akron,DL,75,243.0,4.60,122.0,4.44,0.0
8,"Ferrell, Clelin",Clemson,DL,76,264.0,NaN,NaN,4.40,1.0
9,"Granderson, Carl",Wyoming,DL,77,254.0,4.79,119.0,4.41,0.0


In [86]:
#Testing Data
Name = testdata["name"]
Position = testdata["position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["name","college","position"],1)
testdata.head()

,height_in,weight_lbs,40yards,broad_jump_in,shuttle,allamerican
0,77,262.0,4.63,112.0,4.36,1.0
1,75,250.0,4.62,134.0,4.27,0.0
2,76,266.0,4.79,116.0,4.14,0.0
3,75,252.0,4.65,126.0,NaN,0.0
4,77,249.0,4.53,129.0,NaN,0.0


In [87]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

In [88]:
Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")

In [89]:
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Josh Allen,DL,1
1,Ben Banogu,DL,1
2,Nick Bosa,DL,1
3,Jordan Brailford,DL,1
4,Brian Burns,DL,1
5,Malik Carney,DL,1
6,Maxx Crosby,DL,1
7,Jamal Davis,DL,1
8,Clelin Ferrell,DL,1
9,Carl Granderson,DL,1


In [91]:
#3 Offensive Line
df = pd.read_csv("../OL/OL-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

X["benchpress"] = X["benchpress"].fillna(value=0)
X["shuttle"] = X["shuttle"].fillna(value=100)
X["3cone"] = X["3cone"].fillna(value=100)
X["40yards"] = X["40yards"].fillna(value=100)

X = X.drop([item for item in X.columns.tolist() if item not in ['40yards', 'weight_lbs', 'benchpress', '3cone', 'shuttle',"wonaward"]], 1)

# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier(learning_rate=.1,n_estimators=100,max_depth=3,min_child_weight=3,gamma=0.1,colsample_bytree=.7)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 71.34%


In [96]:
#1 Offensive line
testdata = pd.read_csv("OL-Test.csv")
testdata

,name,college,position,weight_lbs,40yards,benchpress,shuttle,3cone,wonaward
0,"Adams, Paul",Missouri,OL,317.0,5.18,16.0,4.74,7.68,NaN
1,"Bailey, Zack",South Carolina,OL,299.0,NaN,24.0,NaN,NaN,NaN
2,"Barton, Jackson",Utah,OL,310.0,5.18,30.0,4.03,6.90,NaN
3,"Bates, Ryan",Penn St.,OL,306.0,5.09,28.0,4.53,7.45,NaN
4,"Benzschawel, Beau",Wisconsin,OL,309.0,5.24,20.0,NaN,NaN,NaN
5,"Bradbury, Garrett",NC State,OL,306.0,4.92,34.0,4.53,7.41,NaN
6,"Cajuste, Yodny",West Virginia,OL,312.0,NaN,32.0,NaN,NaN,NaN
7,"Daley, Dennis",South Carolina,OL,317.0,5.23,20.0,4.92,7.95,NaN
8,"Davis, Nate",Charlotte,OL,316.0,5.23,23.0,4.83,7.94,NaN
9,"Deiter, Michael",Wisconsin,OL,309.0,5.23,21.0,4.81,7.88,NaN


In [94]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [97]:
for index, row in testdata.iterrows():
    if (testdata.loc[index,"wonaward"] != 1.0):
        if (testdata.loc[index,"wonaward"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if (len(browser.find_by_css("h1[itemprop='name']")) > 0):
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    testdata.loc[index,"wonaward"] = 1
                else:
                    testdata.loc[index,"wonaward"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                testdata.loc[index,"wonaward"] = 0
            else:
                listed = browser.find_by_css("div.search-item")            
                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) > 0):
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                testdata.loc[index,"wonaward"] = 1
                            else:
                                testdata.loc[index,"wonaward"] = 0
                            break
                        else:
                            testdata.loc[index,"wonaward"] = 0
                            break
                    else:
                        testdata.loc[index,"wonaward"] = 0

In [100]:
testdata.to_csv("OL-Test.csv", index=False)

In [105]:
testdata = pd.read_csv("OL-Test.csv")
Name = testdata["Name"]
Position = testdata["Position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["Name","College","Position"],1)
testdata.head()

,weight_lbs,40yards,benchpress,shuttle,3cone,wonaward
0,317.0,5.18,16.0,4.74,7.68,0
1,299.0,NaN,24.0,NaN,NaN,0
2,310.0,5.18,30.0,4.03,6.90,0
3,306.0,5.09,28.0,4.53,7.45,0
4,309.0,5.24,20.0,NaN,NaN,1


In [106]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")
    
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Paul Adams,OL,1
1,Zack Bailey,OL,1
2,Jackson Barton,OL,1
3,Ryan Bates,OL,1
4,Beau Benzschawel,OL,1
5,Garrett Bradbury,OL,1
6,Yodny Cajuste,OL,1
7,Dennis Daley,OL,1
8,Nate Davis,OL,1
9,Michael Deiter,OL,1


In [107]:
#4 Quarterbacks
df = pd.read_csv("../QB/QB-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

X = X.drop([item for item in X.columns.tolist() if item not in ['40yards', 'height_in', 'vert_leap_in', 'broad_jump_in', 'shuttle',"cmp","att","yds","int","heisman","allamerican"]], 1)

# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier(learning_rate=.1,n_estimators=100,max_depth=3,min_child_weight=1,gamma=0,colsample_bytree=1)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.42%


In [112]:
testdata = pd.read_csv("QB-Test.csv")
testdata

,name,college,position,height_in,40yards,vert_leap_in,broad_jump_in,shuttle,cmp,att,yds,int,heisman,allamerican
0,"Browning, Jake",Washington,QB,74,4.74,29.0,112.0,4.44,NaN,NaN,NaN,NaN,NaN,NaN
1,"Finley, Ryan",NC State,QB,76,4.73,30.5,116.0,4.20,NaN,NaN,NaN,NaN,NaN,NaN
2,"Fitzgerald, Nick",Miss. State,QB,77,4.64,29.5,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Grier, Will",West Virginia,QB,74,4.84,34.0,112.0,4.28,NaN,NaN,NaN,NaN,NaN,NaN
4,"Haskins, Dwayne",Ohio St.,QB,75,5.04,28.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Jackson, Tyree",Buffalo,QB,79,4.59,34.5,120.0,4.28,NaN,NaN,NaN,NaN,NaN,NaN
6,"Jones, Daniel",Duke,QB,77,4.81,33.5,120.0,4.41,NaN,NaN,NaN,NaN,NaN,NaN
7,"Lock, Drew",Missouri,QB,76,4.69,31.0,112.0,4.12,NaN,NaN,NaN,NaN,NaN,NaN
8,"McSorley, Trace",Penn St.,QB,72,4.57,33.0,115.0,4.12,NaN,NaN,NaN,NaN,NaN,NaN
9,"Minshew, Gardner",Washington St.,QB,73,4.97,33.5,116.0,4.45,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [115]:
for index, row in testdata.iterrows():
    if (testdata.loc[index,"allamerican"] != 1.0):
        if (testdata.loc[index,"allamerican"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if (len(browser.find_by_css("table#passing")) > 0):
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    testdata.loc[index,"allamerican"] = 1
                else:
                    testdata.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("li.bling_hof")) > 0):
                    testdata.loc[index,"heisman"] = 1
                else:
                    testdata.loc[index,"heisman"] = 0

                testdata.loc[index, "cmp"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_cmp']").text
                testdata.loc[index, "att"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_att']").text
                testdata.loc[index, "yds"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_yds']").text
                testdata.loc[index, "int"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_int']").text
            elif (len(browser.find_by_css("div.search-item")) == 0):
                testdata.loc[index,"allamerican"] = 0
                testdata.loc[index,"heisman"] = 0
            else:
                listed = browser.find_by_css("div.search-item")            
                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("table#passing")) > 0):
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                testdata.loc[index,"allamerican"] = 1
                            else:
                                testdata.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("li.bling_hof")) > 0):
                                testdata.loc[index,"heisman"] = 1
                            else:
                                testdata.loc[index,"heisman"] = 0

                                testdata.loc[index, "cmp"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_cmp']").text
                                testdata.loc[index, "att"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_att']").text
                                testdata.loc[index, "yds"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_yds']").text
                                testdata.loc[index, "int"] = browser.find_by_css("table#passing tfoot tr.bold")[0].find_by_css("td[data-stat='pass_int']").text
                            break
                        else:
                            testdata.loc[index,"allamerican"] = 0
                            testdata.loc[index,"heisman"] = 0
                            break
                    else:
                        testdata.loc[index,"allamerican"] = 0
                        testdata.loc[index,"heisman"] = 0


In [ ]:
testdata.to_csv("QB-Test.csv", index=False)

In [119]:
testdata = pd.read_csv("QB-Test.csv")
Name = testdata["name"]
Position = testdata["position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["name","college","position"],1)
testdata.head()

,height_in,40yards,vert_leap_in,broad_jump_in,shuttle,cmp,att,yds,int,heisman,allamerican
0,74,4.74,29.0,112.0,4.44,958.0,1484.0,12296.0,34.0,0,0
1,76,4.73,30.5,116.0,4.20,938.0,1461.0,11147.0,30.0,0,0
2,77,4.64,29.5,115.0,NaN,511.0,942.0,6207.0,30.0,0,0
3,74,4.84,34.0,112.0,4.28,621.0,945.0,8556.0,23.0,0,0
4,75,5.04,28.5,NaN,NaN,413.0,590.0,5396.0,9.0,0,0


In [120]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")
    
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Jake Browning,QB,1
1,Ryan Finley,QB,1
2,Nick Fitzgerald,QB,1
3,Will Grier,QB,1
4,Dwayne Haskins,QB,0
5,Tyree Jackson,QB,1
6,Daniel Jones,QB,1
7,Drew Lock,QB,1
8,Trace McSorley,QB,1
9,Gardner Minshew,QB,0


In [122]:
#4 Safety
df = pd.read_csv("../S/S-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

X = X.drop([item for item in X.columns.tolist() if item not in ['40yards', "weight_lbs"]], 1)

# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier(learning_rate=.1,n_estimators=100,max_depth=3,min_child_weight=7,gamma=0.4,colsample_bytree=1)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 74.19%


In [124]:
#1 Offensive line
testdata = pd.read_csv("S-Test.csv")
testdata

,name,college,position,weight_lbs,40yards
0,"Abram, Johnathan",Miss. State,S,215,4.45
1,"Amadi, Ugo",NaN,S,201,4.51
2,"Battle, John",LSU,S,201,4.67
3,"Bell, Mike",Fresno St.,S,210,4.83
4,"Blair, Marquise",Utah,S,195,4.48
5,"Crawford, Jonathan",Indiana,S,205,NaN
6,"Denis, Lukas",Boston College,S,190,4.64
7,"Dixon, D'Cota",Wisconsin,S,204,4.81
8,"Gant, Malik",Marshall,S,209,4.63
9,"Gardner-Johnson, Chauncey",Florida,S,210,4.48


In [125]:
Name = testdata["name"]
Position = testdata["position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["name","college","position"],1)
testdata.head()

,weight_lbs,40yards
0,215,4.45
1,201,4.51
2,201,4.67
3,210,4.83
4,195,4.48


In [126]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")
    
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Johnathan Abram,S,1
1,Ugo Amadi,S,1
2,John Battle,S,0
3,Mike Bell,S,0
4,Marquise Blair,S,1
5,Jonathan Crawford,S,1
6,Lukas Denis,S,0
7,D'Cota Dixon,S,0
8,Malik Gant,S,1
9,Chauncey Gardner-Johnson,S,1


In [127]:
#6 Tight Ends
df = pd.read_csv("../TE/TE-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

X = X.drop([item for item in X.columns.tolist() if item not in ['40yards', 'weight_lbs', 'benchpress', 'shuttle',"hasbling", "scrimyds","scrimtds","allamerican"]], 1)

# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier(learning_rate=.001,n_estimators=300,max_depth=11,min_child_weight=3,gamma=0.4,colsample_bytree=.3)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 83.64%


In [128]:
testdata = pd.read_csv("TE-Test.csv")
testdata

,name,college,position,weight_lbs,40yards,benchpress,shuttle,allamerican,hasbling,scrimyds,scrimtds
0,"Blanton, Kendall",Missouri,TE,262,4.95,22.0,4.42,NaN,NaN,NaN,NaN
1,"Brown, Keenen",Texas St.,TE,250,4.75,17.0,4.51,NaN,NaN,NaN,NaN
2,"Fant, Noah",Iowa,TE,249,4.50,20.0,4.22,NaN,NaN,NaN,NaN
3,"Gentry, Zach",Michigan,TE,265,4.90,12.0,4.53,NaN,NaN,NaN,NaN
4,"Helm, Daniel",Duke,TE,249,4.89,14.0,NaN,NaN,NaN,NaN,NaN
5,"Hockenson, T.J.",Iowa,TE,251,4.70,17.0,4.18,NaN,NaN,NaN,NaN
6,"Knox, Dawson",Ole Miss,TE,254,NaN,16.0,4.27,NaN,NaN,NaN,NaN
7,"Mack, Alize",Notre Dame,TE,249,4.70,22.0,4.34,NaN,NaN,NaN,NaN
8,"Moreau, Foster",LSU,TE,253,4.66,22.0,4.11,NaN,NaN,NaN,NaN
9,"Nauta, Isaac",Georgia,TE,244,4.91,19.0,4.43,NaN,NaN,NaN,NaN


In [129]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [130]:
for index, row in testdata.iterrows():
    if (testdata.loc[index,"hasbling"] != 1.0):
        if (testdata.loc[index,"hasbling"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                try:
                    testdata.loc[index, "scrimyds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds']").text
                    testdata.loc[index, "scrimtds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_td']").text
                except:
                    testdata.loc[index, "scrimyds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds']").text
                    testdata.loc[index, "scrimtds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_td']").text
                            
                if (len(browser.find_by_css("ul#bling")) > 0):
                    testdata.loc[index,"hasbling"] = 1
                else:
                    testdata.loc[index,"hasbling"] = 0
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    testdata.loc[index,"allamerican"] = 1
                else:
                    testdata.loc[index,"allamerican"] = 0
                
            elif (len(browser.find_by_css("div.search-item")) == 0):
                testdata.loc[index,"hasbling"] = 0
                testdata.loc[index,"allamerican"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            try:
                                testdata.loc[index, "scrimyds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds']").text
                                testdata.loc[index, "scrimtds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_td']").text
                            except:
                                testdata.loc[index, "scrimyds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds']").text
                                testdata.loc[index, "scrimtds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_td']").text
                            
                            if (len(browser.find_by_css("ul#bling")) > 0):
                                testdata.loc[index,"hasbling"] = 1
                            else:
                                testdata.loc[index,"hasbling"] = 0
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                testdata.loc[index,"allamerican"] = 1
                            else:
                                testdata.loc[index,"allamerican"] = 0
                            break
                        else:
                            testdata.loc[index,"hasbling"] = 0
                            testdata.loc[index,"allamerican"] = 0
                            break
                    else:
                        testdata.loc[index,"hasbling"] = 0
                        testdata.loc[index,"allamerican"] = 0

In [131]:
testdata.to_csv("TE-Test.csv", index=False)

In [132]:
testdata

,name,college,position,weight_lbs,40yards,benchpress,shuttle,allamerican,hasbling,scrimyds,scrimtds
0,"Blanton, Kendall",Missouri,TE,262,4.95,22.0,4.42,0.0,0.0,476,6
1,"Brown, Keenen",Texas St.,TE,250,4.75,17.0,4.51,0.0,0.0,755,7
2,"Fant, Noah",Iowa,TE,249,4.50,20.0,4.22,0.0,0.0,1083,19
3,"Gentry, Zach",Michigan,TE,265,4.90,12.0,4.53,0.0,0.0,817,4
4,"Helm, Daniel",Duke,TE,249,4.89,14.0,NaN,0.0,0.0,804,6
5,"Hockenson, T.J.",Iowa,TE,251,4.70,17.0,4.18,0.0,0.0,1084,10
6,"Knox, Dawson",Ole Miss,TE,254,NaN,16.0,4.27,0.0,0.0,605,0
7,"Mack, Alize",Notre Dame,TE,249,4.70,22.0,4.34,0.0,0.0,716,4
8,"Moreau, Foster",LSU,TE,253,4.66,22.0,4.11,0.0,0.0,629,6
9,"Nauta, Isaac",Georgia,TE,244,4.91,19.0,4.43,0.0,0.0,936,9


In [136]:
testdata = pd.read_csv("TE-Test.csv")
Name = testdata["name"]
Position = testdata["position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["name","college","position"],1)
testdata.head()

,weight_lbs,40yards,benchpress,shuttle,allamerican,hasbling,scrimyds,scrimtds
0,262,4.95,22.0,4.42,0.0,0.0,476,6
1,250,4.75,17.0,4.51,0.0,0.0,755,7
2,249,4.50,20.0,4.22,0.0,0.0,1083,19
3,265,4.90,12.0,4.53,0.0,0.0,817,4
4,249,4.89,14.0,NaN,0.0,0.0,804,6


In [137]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")
    
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Kendall Blanton,TE,1
1,Keenen Brown,TE,1
2,Noah Fant,TE,1
3,Zach Gentry,TE,0
4,Daniel Helm,TE,1
5,T.J. Hockenson,TE,1
6,Dawson Knox,TE,1
7,Alize Mack,TE,1
8,Foster Moreau,TE,1
9,Isaac Nauta,TE,0


In [138]:
#4 Cornerbacks
df = pd.read_csv("../CB/CB-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

X = X.drop([item for item in X.columns.tolist() if item not in ['40yards', 'height_in', 'vert_leap_in', '3cone', 'defint',"deftds","hasbling","allamerican","hasawards"]], 1)

# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier(learning_rate=.1,n_estimators=200,max_depth=2,min_child_weight=1,gamma=0.1,colsample_bytree=.3)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.28%


In [148]:
testdata = pd.read_csv("CB-Test.csv")
testdata

,name,college,position,height_in,40yards,vert_leap_in,3cone,defint,deftds,hasbling,allamerican,hasawards
0,"Austin, Blessuan",Rutgers,CB,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Baity, Derrick",Kentucky,CB,74,NaN,38.5,7.06,NaN,NaN,NaN,NaN,NaN
2,"Baker, Deandre",Georgia,CB,71,4.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ballentine, Corey",Washburn,CB,72,4.47,39.5,6.82,NaN,NaN,NaN,NaN,NaN
4,"Boyd, Kris",Texas,CB,71,4.45,36.5,6.94,NaN,NaN,NaN,NaN,NaN
5,"Brown, Jordan",South Dakota State,CB,72,4.51,39.5,NaN,NaN,NaN,NaN,NaN,NaN
6,"Brown, Blace",Troy,CB,72,4.75,NaN,7.33,NaN,NaN,NaN,NaN,NaN
7,"Bunting, Sean",C. Michigan,CB,72,4.42,41.5,NaN,NaN,NaN,NaN,NaN,NaN
8,"Cheevers, Hamp",Boston College,CB,69,4.52,39.0,6.96,NaN,NaN,NaN,NaN,NaN
9,"Crawford, Xavier",C. Michigan,CB,71,4.48,37.5,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
for index, row in testdata.iterrows():
    if (testdata.loc[index,"hasbling"] != 1.0):
        if (testdata.loc[index,"hasbling"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                try:
                    testdata.loc[index, "defint"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int']").text
                except:
                    testdata.loc[index, "defint"] = np.nan

                try:
                    testdata.loc[index, "deftds"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int_td']").text
                except:
                    testdata.loc[index, "deftds"] = np.nan
                    
                if (len(browser.find_by_css("ul#bling")) > 0):
                    testdata.loc[index,"hasbling"] = 1
                else:
                    testdata.loc[index,"hasbling"] = 0
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    testdata.loc[index,"allamerican"] = 1
                else:
                    testdata.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    testdata.loc[index,"hasawards"] = 1
                else:
                    testdata.loc[index,"hasawards"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                testdata.loc[index,"hasbling"] = 0
                testdata.loc[index,"allamerican"] = 0
                testdata.loc[index,"hasawards"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            try:
                                testdata.loc[index, "defint"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int']").text
                            except:
                                testdata.loc[index, "defint"] = np.nan
                                
                            try:
                                testdata.loc[index, "deftds"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int_td']").text
                            except:
                                testdata.loc[index, "deftds"] = np.nan

                            if (len(browser.find_by_css("ul#bling")) > 0):
                                testdata.loc[index,"hasbling"] = 1
                            else:
                                testdata.loc[index,"hasbling"] = 0
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                testdata.loc[index,"allamerican"] = 1
                            else:
                                testdata.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                testdata.loc[index,"hasawards"] = 1
                            else:
                                testdata.loc[index,"hasawards"] = 0
                            break
                        else:
                            testdata.loc[index,"hasbling"] = 0
                            testdata.loc[index,"allamerican"] = 0
                            testdata.loc[index,"hasawards"] = 0
                            break
                    else:
                        testdata.loc[index,"hasbling"] = 0
                        testdata.loc[index,"allamerican"] = 0
                        testdata.loc[index,"hasawards"] = 0

In [151]:
testdata.to_csv("CB-Test.csv", index=False)

In [152]:
testdata = pd.read_csv("CB-Test.csv")
Name = testdata["name"]
Position = testdata["position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["name","college","position"],1)
testdata.head()

,height_in,40yards,vert_leap_in,3cone,defint,deftds,hasbling,allamerican,hasawards
0,73,NaN,NaN,NaN,4,1,0,0,0
1,74,NaN,38.5,7.06,6,0,0,0,0
2,71,4.52,NaN,NaN,7,0,1,1,1
3,72,4.47,39.5,6.82,0,0,0,0,0
4,71,4.45,36.5,6.94,3,0,0,0,0


In [153]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")
    
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Blessuan Austin,CB,1
1,Derrick Baity,CB,1
2,Deandre Baker,CB,1
3,Corey Ballentine,CB,1
4,Kris Boyd,CB,1
5,Jordan Brown,CB,1
6,Blace Brown,CB,0
7,Sean Bunting,CB,1
8,Hamp Cheevers,CB,1
9,Xavier Crawford,CB,1


In [164]:
#6 Tight Ends
df = pd.read_csv("../RB/RFBacks-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

X = X.drop([item for item in X.columns.tolist() if item not in ['40yards', 'weight_lbs', 'benchpress', 'shuttle',"rushatt", "rushyds","rushypa","rushtds","allamerican","heisman"]], 1)

# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier(learning_rate=.01,n_estimators=200,max_depth=9,min_child_weight=5,gamma=0.2,colsample_bytree=1)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 69.63%


In [165]:
testdata = pd.read_csv("RB-Test.csv")
testdata

,name,college,position,weight_lbs,40yards,benchpress,shuttle,rushatt,rushyds,rushypa,rushtds,allamerican,heisman
0,"Anderson, Rodney",Oklahoma,RB,224,NaN,25.0,NaN,200.0,1285.0,6.4,16.0,0.0,0.0
1,"Armstead, Ryquell",Temple,RB,220,4.45,22.0,4.29,573.0,2812.0,4.9,34.0,0.0,0.0
2,"Barnes, Alex",Kansas St.,RB,226,4.59,34.0,4.10,458.0,2616.0,5.7,25.0,0.0,0.0
3,"Brossette, Nick",LSU,RB,209,4.72,15.0,4.44,286.0,1345.0,4.7,14.0,0.0,0.0
4,"Gaskin, Myles",Washington,RB,205,4.58,24.0,4.27,945.0,5323.0,5.6,57.0,0.0,0.0
5,"Harris, Damien",Alabama,RB,216,4.57,16.0,NaN,477.0,3070.0,6.4,23.0,0.0,0.0
6,"Henderson, Darrell",Memphis,RB,208,4.49,22.0,NaN,431.0,3545.0,8.2,36.0,1.0,0.0
7,"Higdon, Karan",Michigan,RB,206,4.49,21.0,NaN,471.0,2616.0,5.6,27.0,0.0,0.0
8,"Hill, Justice",Oklahoma St.,RB,198,4.40,21.0,NaN,632.0,3539.0,5.6,30.0,0.0,0.0
9,"Holyfield, Elijah",Georgia,RB,217,4.78,26.0,NaN,215.0,1340.0,6.2,9.0,0.0,0.0


In [157]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [159]:
for index, row in testdata.iterrows():
    if (testdata.loc[index,"allamerican"] != 1.0):
        if (testdata.loc[index,"allamerican"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                try:
                    testdata.loc[index, "rushatt"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_att']").text
                    testdata.loc[index, "rushyds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds']").text
                    testdata.loc[index, "rushypa"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds_per_att']").text
                    testdata.loc[index, "rushtds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_td']").text
                except:
                    testdata.loc[index, "rushatt"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_att']").text
                    testdata.loc[index, "rushyds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds']").text
                    testdata.loc[index, "rushypa"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds_per_att']").text
                    testdata.loc[index, "rushtds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_td']").text
                            
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    testdata.loc[index,"allamerican"] = 1
                else:
                    testdata.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("li.bling_hof")) > 0):
                    testdata.loc[index,"heisman"] = 1
                else:
                    testdata.loc[index,"heisman"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                testdata.loc[index,"allamerican"] = 0
                testdata.loc[index,"heisman"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            try:
                                testdata.loc[index, "rushatt"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_att']").text
                                testdata.loc[index, "rushyds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds']").text
                                testdata.loc[index, "rushypa"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds_per_att']").text
                                testdata.loc[index, "rushtds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='rush_td']").text
                            except:
                                testdata.loc[index, "rushatt"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_att']").text
                                testdata.loc[index, "rushyds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds']").text
                                testdata.loc[index, "rushypa"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_yds_per_att']").text
                                testdata.loc[index, "rushtds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='rush_td']").text

                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                testdata.loc[index,"allamerican"] = 1
                            else:
                                testdata.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("li.bling_hof")) > 0):
                                testdata.loc[index,"heisman"] = 1
                            else:
                                testdata.loc[index,"heisman"] = 0
                            break
                        else:
                            testdata.loc[index,"allamerican"] = 0
                            testdata.loc[index,"heisman"] = 0
                            break
                    else:
                        testdata.loc[index,"allamerican"] = 0
                        testdata.loc[index,"heisman"] = 0


In [160]:
testdata.to_csv("RB-Test.csv", index=False)

In [167]:
testdata = pd.read_csv("RB-Test.csv")
Name = testdata["name"]
Position = testdata["position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["name","college","position"],1)
testdata.head()

,weight_lbs,40yards,benchpress,shuttle,rushatt,rushyds,rushypa,rushtds,allamerican,heisman
0,224,NaN,25.0,NaN,200.0,1285.0,6.4,16.0,0.0,0.0
1,220,4.45,22.0,4.29,573.0,2812.0,4.9,34.0,0.0,0.0
2,226,4.59,34.0,4.10,458.0,2616.0,5.7,25.0,0.0,0.0
3,209,4.72,15.0,4.44,286.0,1345.0,4.7,14.0,0.0,0.0
4,205,4.58,24.0,4.27,945.0,5323.0,5.6,57.0,0.0,0.0


In [168]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")
    
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Rodney Anderson,RB,1
1,Ryquell Armstead,RB,1
2,Alex Barnes,RB,1
3,Nick Brossette,RB,0
4,Myles Gaskin,RB,1
5,Damien Harris,RB,1
6,Darrell Henderson,RB,1
7,Karan Higdon,RB,1
8,Justice Hill,RB,1
9,Elijah Holyfield,RB,1


In [169]:
#6 Tight Ends
df = pd.read_csv("../P/P-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

X = X.drop([item for item in X.columns.tolist() if item not in ['40yards', 'weight_lbs', 'height_in']], 1)

# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 60.00%


In [170]:
testdata = pd.read_csv("P-Test.csv")
testdata

,name,college,position,height_in,weight_lbs,40yards
0,"Bailey, Jake",Stanford,P,73,200,4.72
1,"Fox, Jack",Rice,P,74,213,4.73
2,"Wishnowsky, Mitch",Utah,P,74,218,4.63


In [171]:
#Testing Data
Name = testdata["name"]
Position = testdata["position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["name","college","position"],1)
testdata.head()

,height_in,weight_lbs,40yards
0,73,200,4.72
1,74,213,4.73
2,74,218,4.63


In [172]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")
    
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Jake Bailey,P,0
1,Jack Fox,P,0
2,Mitch Wishnowsky,P,0


In [177]:
#6 Tight Ends
df = pd.read_csv("../LB/Dbacks_Train.csv")
df = df.drop(["year","name","college","position"],1)
df.head()
# 721

,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,hasawards,allamerican,drafted
0,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60,0,0,N
1,74.6,239,4.92,18.0,33.5,115.0,4.28,7.48,0,0,N
2,72.9,261,4.92,15.0,33.0,109.0,4.42,7.88,0,0,N
3,73.0,230,4.92,NaN,32.5,115.0,4.29,7.67,0,0,N
4,69.6,215,4.52,21.0,38.0,125.0,4.21,7.48,0,0,Y


In [178]:
Y = df["drafted"]
Y = Y.replace("Y",1)
Y = Y.replace("N",0)

#X = df.drop(['weight_lbs', '3cone', 'Crec', 'Cyds', 'hasbling', 'hasawards', "drafted"], 1)
X = df.drop(["drafted"],1)

X = X.drop([item for item in X.columns.tolist() if item in ['weight_lbs', 'benchpress', 'broad_jump_in', 'shuttle', 'allamerican']], 1)

# split data into train and test sets
seed = 777
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# fit model on training data
model = XGBClassifier(learning_rate=.1,n_estimators=100,max_depth=2,min_child_weight=3,gamma=0,colsample_bytree=0.5)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 78.88%


In [187]:
testdata = pd.read_csv("LB-Test.csv")
testdata.head()

,name,college,position,height_in,40yards,vert_leap_in,3cone,hasawards
0,"Al-Shaair, Azeez",FAU,LB,73,NaN,NaN,NaN,NaN
1,"Alaka, Otaro",Texas A&M,LB,75,4.82,36.0,NaN,NaN
2,"Allen, Dakota",Texas Tech,LB,73,4.77,NaN,7.15,NaN
3,"Allen-Williams, Bryson",South Carolina,LB,73,4.88,NaN,7.40,NaN
4,"Allison, Jeff",Fresno St.,LB,71,4.82,31.0,7.20,NaN


In [188]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [193]:
for index, row in testdata.iterrows():
    if (testdata.loc[index,"hasawards"] != 1.0):
        if (testdata.loc[index,"hasawards"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    testdata.loc[index,"hasawards"] = 1
                else:
                    testdata.loc[index,"hasawards"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                testdata.loc[index,"hasawards"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                testdata.loc[index,"hasawards"] = 1
                            else:
                                testdata.loc[index,"hasawards"] = 0
                        else:
                            testdata.loc[index,"hasawards"] = 0
                            break
                    else:
                        testdata.loc[index,"hasawards"] = 0

In [194]:
testdata.to_csv("LB-Test.csv", index=False)

In [195]:
#Testing Data
Name = testdata["name"]
Position = testdata["position"]
#df = df.drop(["year","name","college","position"],1)
testdata = testdata.drop(["name","college","position"],1)
testdata.head()

,height_in,40yards,vert_leap_in,3cone,hasawards
0,73,NaN,NaN,NaN,0.0
1,75,4.82,36.0,NaN,0.0
2,73,4.77,NaN,7.15,0.0
3,73,4.88,NaN,7.40,0.0
4,71,4.82,31.0,7.20,1.0


In [196]:
# make predictions for test data
y_pred = model.predict(testdata)
predictions = [round(value) for value in y_pred]

Names = []
for name in Name:
    name = name.split(",")
    Names.append(f"{name[1]} {name[0]}")
    
predictors = pd.DataFrame({"Name":Names,"Position":Position,"finalmodel":predictions})
predictors

,Name,Position,finalmodel
0,Azeez Al-Shaair,LB,1
1,Otaro Alaka,LB,1
2,Dakota Allen,LB,0
3,Bryson Allen-Williams,LB,0
4,Jeff Allison,LB,1
5,Cody Barton,LB,1
6,Ben Burr-Kirven,LB,1
7,Devin Bush,LB,1
8,Blake Cashman,LB,1
9,Ryan Connelly,LB,1
